# conds

In [ ]:
import numpy as np
import kwant
from numpy import sqrt,pi,kron
import matplotlib.pyplot as plt
import time
from numpy import sqrt,pi,cos,sin,kron
from scipy.sparse.linalg import eigsh

%matplotlib tk

#pauli matrix
sigma_0 = np.array([[1, 0],[0, 1]])
sigma_x = np.array([[0, 1],[1, 0]])
sigma_y = np.array([[0, -1j],[1j, 0]])
sigma_z = np.array([[1, 0],[0, -1]])

tau_0 = sigma_0
tau_x = sigma_x
tau_y = sigma_y
tau_z = sigma_z

L = 300
delta = 1
tw = 12
ts = 6
mu_s = 0
Vs = 0
alpha = 0.8
T = 0.4



def BdG_conds_1D(Vw, mu_w, t_lead = tw, mu_lead = 2*tw):
    sys = kwant.Builder()
    wire_lat = kwant.lattice.square(a=1,norbs = 4, name = 'wire')
    sc_lat = kwant.lattice.square(a=1,norbs = 4,name = 'sc')

    #wire onsite
    # for i in range(L+1):
    #     sys[ wire_lat(i, 0) ] = (2 * tw - mu_w) * kron(tau_z, sigma_0) + Vw * kron(tau_0, sigma_x)

    sys[ (wire_lat(i,0) for i in range(L+1))] = (2 * tw - mu_w) * kron(tau_z, sigma_0) + Vw * kron(tau_0, sigma_x)

    #wire hopping
    sys[kwant.builder.HoppingKind((1, 0), wire_lat)] = -tw * kron(tau_z, sigma_0) + 1j * alpha / 2 * kron(tau_z, sigma_y)


    #left lead
    left = kwant.Builder(kwant.TranslationalSymmetry((-1,0)))
    left[wire_lat(0, 0)] =  (2 * t_lead-mu_lead) * kron(tau_z,sigma_0)
    left[kwant.builder.HoppingKind((-1, 0), wire_lat)] = -t_lead * kron(tau_z,sigma_0)
    sys.attach_lead(left)

    #rignt lead
    rignt = kwant.Builder(kwant.TranslationalSymmetry((1,0)))
    rignt[wire_lat(L, 0)] =  (2 * t_lead-mu_lead) * kron(tau_z,sigma_0)
    rignt[kwant.builder.HoppingKind((-1, 0), wire_lat)] = -t_lead * kron(tau_z,sigma_0)
    sys.attach_lead(rignt)

    #sc
    sys[ (sc_lat(i,-1) for i in range(L+1))] = (2 * ts - mu_s) * kron(tau_z, sigma_0) + delta * kron(tau_x, sigma_0) + Vs * kron(tau_0, sigma_x)
    sys[kwant.builder.HoppingKind((1,0),sc_lat)] = -ts * kron(tau_z, sigma_0) 

    #tunneling 
    sys[kwant.builder.HoppingKind((0,1),wire_lat,sc_lat)] = T * kron(tau_z,sigma_0)
    return sys




def local_cond(S):
    ree = np.sum(np.abs(S[:len(S)//2, :len(S)//2])**2)
    reh = np.sum(np.abs(S[:len(S)//2, len(S)//2:])**2)
    return (len(S)//2 - ree + reh)


def nonlocal_cond(S):
    ree = np.sum(np.abs(S[:len(S)//2, :len(S)//2])**2)
    reh = np.sum(np.abs(S[:len(S)//2, len(S)//2:])**2)
    return (ree - reh)


def cal_conds(sys,energys):
    smatrix = kwant.smatrix(sys,energy=energys,check_hermiticity=True)
    GLL=local_cond(np.array(smatrix.submatrix(0,0)))
    GLR=nonlocal_cond(np.array(smatrix.submatrix(0,1)))
    return GLL,GLR

def plot_cond(x,y,GLL,GLR):
    X, Y = np.meshgrid(x, y)
    Z1 = np.array(GLL).T
    Z2 = np.array(GLR).T

    plt.figure(figsize=((12,4)))
    plt.suptitle(f"L={L}", fontsize=16)

    plt.subplot(1,2,1)
    plt.pcolormesh(X, Y, Z1, cmap='bwr', shading='auto') 
    plt.xlabel('B')
    plt.ylabel('bias')
    cbar = plt.colorbar()  

    plt.subplot(1,2,2)
    plt.pcolormesh(X, Y, Z2, cmap='bwr', shading='auto') 
    plt.xlabel('B')
    cbar = plt.colorbar() 

    plt.show()






# 1D conds

In [13]:
Vw = 0.2
mu_w = 0.1
bias_range = np.linspace(-0.1,0.1,201)


system = BdG_conds_1D(Vw, mu_w)
system = system.finalized()
kwant.plot(system)

# conds
t1 = time.time()
GLL = np.zeros(len(bias_range))
GLR = np.zeros(len(bias_range))

for i in range(len(bias_range)):
    GLL[i], GLR[i] = cal_conds(system,bias_range[i])

t2 = time.time()
print(t2-t1)

plt.figure(figsize=(10,4))
plt.suptitle(f'$V_w$={Vw},$\\mu_w$={mu_w},$\\Delta$={delta}', fontsize=16)
plt.subplot(1,2,1)
plt.plot(bias_range, GLL, label="GLL")
plt.xlabel('bias')
plt.ylabel('e^2/h')

plt.subplot(1,2,2)
plt.plot(bias_range, GLR, label="GLR")
plt.xlabel('bias')
plt.ylabel('e^2/h')
plt.show()

3.7396960258483887


# 2D conds

In [12]:
# import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

Vw_range = np.linspace(0,0.3,31)
bias_range = np.linspace(-0.1,0.1,41)

t1=time.time()
GLL=np.zeros((len(Vw_range),len(bias_range)))
GLR=np.zeros((len(Vw_range),len(bias_range)))

for i in range(len(Vw_range)):
    for j in range(len(bias_range)):
        system = BdG_conds_1D(Vw_range[i],mu_w = 0.1).finalized()
        GLL[i][j], GLR[i][j] = cal_conds(system,bias_range[j])

t2=time.time()
print(t2-t1)

plot_cond(Vw_range,bias_range,GLL,GLR)


def plot_cond(x, y, GLL, GLR):
    X, Y = np.meshgrid(x, y)
    Z1 = np.array(GLL).T
    Z2 = np.array(GLR).T

    plt.figure(figsize=(12, 4))
    plt.suptitle(f"L={L}", fontsize=16)

    norm = mcolors.TwoSlopeNorm(vmin=Z2.min(), vcenter=0, vmax=Z2.max())  # 设置0值为白色

    plt.subplot(1, 2, 1)
    plt.pcolormesh(X, Y, Z1, cmap='bwr', shading='auto')
    plt.xlabel('B')
    plt.ylabel('bias')
    plt.colorbar()

    plt.subplot(1, 2, 2)
    plt.pcolormesh(X, Y, Z2, cmap='bwr', shading='auto', vmin=-0.01, vmax=0.01)  # 应用norm
    plt.xlabel('B')
    plt.colorbar()


plot_cond(Vw_range,bias_range,GLL,GLR)

34.70249056816101
